In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import requests
import os
import pandas as pd
from datetime import datetime, timedelta
import dateparser 
import re

base_url  = 'https://blitzindiamedia.com/'
country = 'bangladesh'
page =1
initial_url = f'{base_url}page/{page}/?s={country}'

# FINDING NUMBER OF PAGES

page = 1
while True:
    
    initial_url = initial_url = f'{base_url}page/{page}/?s={country}'
    response = requests.get(initial_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    page_header = soup.find('div', class_ = 'jnews_search_content_wrapper')
    
    next_page_button = page_header.find('a', class_ = 'page_nav next')
    
    if next_page_button:
        page+=1
    else:
        break
        
print(page)


#COLLECTING ALL LINKS

links = []

pages = [1,2]

for page in pages:
    
    initial_url = f'{base_url}page/{page}/?s={country}'    
    
    response = requests.get(initial_url)
     
    soup = BeautifulSoup(response.text, 'html.parser')

    news_urls = soup.find_all('article', class_ = 'jeg_post jeg_pl_lg_2 format-standard')

    for news in news_urls:
        temp = news.find('a', href = True)
        links.append(temp['href'])

#SCRAPING THE LINKS

counter = 0
data_list = []

for link in links:
    
    country = 'bangladesh'
        
    response = requests.get(link)

    soup = BeautifulSoup(response.text, 'html.parser')

    title_div = soup.find('div', class_ = 'post-wrapper')

    title_tag = title_div.find('h1')
    
    title = title_tag.text if title_tag else 'Title Not Found'

    date_info = soup.find('meta', {'property': 'article:modified_time'})
    

    if date_info:

        date_data = date_info.get('content')
        
        only_date = date_data.split('T')[0]
        only_time = date_data.split('T')[1]
        time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
        cleaned_date = f"{only_date},{time}"
        
        source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M")
        bangladesh_localtime = source_localtime + timedelta(minutes=30)

    else:
        date_data = 'Date Data Not Found'
    
    
    author_tag = soup.find('div', class_ = 'jeg_meta_author')
    author = author_tag.find('a').text if author_tag else 'Author not found'
    
    
    content_header = soup.find('div', class_ = 'entry-content no-share')
    content = []

    if content_header:

        content_main = content_header.find('div', class_ = 'content-inner')
        main_content = content_main.find_all('p')
        for con in main_content:           
            content.append(con.text)        
    else:
        content.append('Content Not Found')


    full_content = ' '.join(content)
    


    
    
    temp_cuts = full_content.split('NEW DELHI', 1)

    if len(temp_cuts) > 1:
        full_content = temp_cuts[1].strip()
    else:
    
        full_content = temp_cuts[0].strip()
    
    
    content_summary = 'Content summary not found'
    
    title_translation = 'None'
    summary_translation = 'None'
    content_translation = 'None'
    
    
    data_dict = {
            "url": link,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }
    

    counter+=1
    

    if (date_data != "Date Data Not Found" and full_content != "Content Not Found"):
        if data_dict not in data_list:
            # Adding to data list
            data_list.append(data_dict)
            print(f'Link {counter} added')
    else:
        print(counter)
        print('Skipped due to missing info.')

df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_Blitz_India_Media.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["date"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df["date"] = pd.to_datetime(df["date"], format = "%d-%m-%Y")  # Converting the "date" column to datetime
    df = df.sort_values(by="date", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
Link 12 added
Link 13 added
Link 14 added
Link 15 added
Link 16 added
Link 17 added
Link 18 added
Link 19 added
Link 20 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://blitzindiamedia.com/bangladesh-ministe...,Bangladesh minister applauds ‘excellent’ relat...,"BANGLADESH’S Minister of Foreign Affairs, AK A...",Content summary not found,None,None,None,Blitz India Media,bangladesh,2024-01-11 14:07:00,2024-01-11 14:37:00
1,https://blitzindiamedia.com/pm-sheikh-hasina-s...,PM Sheikh Hasina says India a great friend of ...,: Prime Minister Sheikh Hasina on January 8 sa...,Content summary not found,None,None,None,Blitz India Media,bangladesh,2024-01-08 12:50:00,2024-01-08 13:20:00
2,https://blitzindiamedia.com/nobel-laureate-yun...,Nobel laureate Yunus sentenced to six months i...,: Bangladesh’s Noble laureate economist Dr Muh...,Content summary not found,None,None,None,Blitz India Media,bangladesh,2024-01-02 12:17:00,2024-01-02 12:47:00
3,https://blitzindiamedia.com/bangladesh-a-dampe...,Bangladesh A dampened Durga Puja,DURGA Puja in Bangladesh observed with great f...,Content summary not found,None,None,None,Blitz India Media,bangladesh,2023-11-09 12:58:00,2023-11-09 13:28:00
4,https://blitzindiamedia.com/india-bangladesh-t...,India-Bangladesh trade through Burimari and Hi...,: In view of Durga Puja festival being celebr...,Content summary not found,None,None,None,Blitz India Media,bangladesh,2023-10-17 08:20:00,2023-10-17 08:50:00
